# (Naive) Growth accounting

This is an example to show you how one can use existing library to access data easily. The growth accounting framework used is naive and the results should be taken with a pinch of salt.

In [ ]:
import py4macro
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

This example relies heavily on `py4macro` which is itself an excellent resource to learn `Python`: https://py4macro.github.io

In [ ]:
help(py4macro.data)

The data we need can be found by Penn World Table. Let's look at the data.

In [ ]:
py4macro.data('pwt',description=1)

Retrieve the data.

In [ ]:
df = py4macro.data('pwt'); print(type(df))

A quick glance of the data.

In [ ]:
df.head()

Print out all countries available

In [ ]:
country_list = df.loc[:,'country'].unique()
country_list

In [ ]:
df.loc[:,'country'].nunique()

Print out all years available

In [ ]:
year_list = df.loc[:,'year'].unique()
year_list

Okay there are quite a few countries and years in the dataset. To make life easier, why not start with a small subset.

In [ ]:
# 1
country_table = ['Japan', 'United Kingdom','United States', 'Norway',
                'Mexico','Peru','India','China','Zimbabwe','Niger']

# 2
cond = (df['country'].isin(country_table)) & (df['year'].isin([2018, 2019]))

# 3
col = ['year', 'country','rgdpo', 'ck', 'labsh','emp']

# 4
table2019 = df.loc[cond,col].set_index('country').sort_values(['country', 'year'])

In [ ]:
table2019

We want to turn each `rgdpo`, `ck`, and `emp` into growth rates and we can write a for-loop to do it all in one go.

In [ ]:
variables = col[2:]
for variable in variables:
    table2019[f'{variable}_g'] = table2019.groupby('country')[variable].pct_change() * 100

In [ ]:
# drop na will allow us to drop all 2018 observations
table2019 = table2019.dropna()

# calcuate the capital share as 1 - labour share
table2019['alpha'] = 1 - table2019['labsh']

# calcuate the growth of factors
table2019['factors_growth'] = table2019['alpha'] * table2019['ck_g'] + table2019['labsh'] * table2019['emp_g']

# the discrepancies will be taken as the Solow residuals
table2019['solow_residual'] = table2019['rgdpo_g'] - table2019['factors_growth']

# sorting the table according to real GDP growth
table2019 = table2019.sort_values(['rgdpo_g'], ascending=False)
table2019.rename(columns={'factors_growth': 'Factor growth', 'solow_residual': 'Solow Residuals'}, inplace=True)

In [ ]:
# plot it
table2019.iloc[:,[-1,-2]].plot(kind='bar', stacked=True)
plt.title("Naive growth accounting")
plt.axhline(y=0, color='r', linestyle='--')

plt.xlabel('Country', fontsize=12)
plt.ylabel('Percentage change in 2019', fontsize=12)

#plt.savefig("naive_growth_accounting.png")